## verion mismatches run this first in terminal
1️⃣ Create and activate a new virtual environment
/opt/homebrew/opt/python@3.11/bin/python3.11 -m venv ~/py311_pandas_env
source ~/py311_pandas_env/bin/activate

2️⃣ Upgrade pip

pip install --upgrade pip

3️⃣ Install Pandas 2.3.1, Jupyter, and compatible libraries

pip install pandas==2.3.1 numpy==1.26 jupyter matplotlib scipy scikit-learn numba explainerdashboard

4️⃣ Launch Jupyter Notebook from this environment

jupyter notebook


In [ ]:
## bioclimatic data found here: https://www.worldclim.org/data/v1.4/worldclim14.html 

In [1]:
import pandas as pd
print(pd.__version__)  # should show 2.3.1



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/homebrew/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/homebrew/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/opt/homebrew/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()
  File "/opt/homebrew/lib/python3.11/site-pack

AttributeError: _ARRAY_API not found

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [1]:
## adding more climate data

import rioxarray as rxr

# Open a .bil file (it automatically reads .hdr)
raster = rxr.open_rasterio("/Users/inesschwartz/GreenDataScience/Thesis/climaticdata/bioclimaticvariables/bio_10m_bil", masked=True)
print(raster)

# each file looks like this bio1.bil-bio19.bil


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/homebrew/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/homebrew/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/opt/homebrew/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()
  File "/opt/homebrew/lib/python3.11/site-pack

AttributeError: _ARRAY_API not found

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
#/Users/inesschwartz/Desktop/GIS_Angola/tiles/ao.shp

In [1]:
## load angola boundaries
import geopandas as gpd

# Load Angola shapefile (must be in WGS84 / EPSG:4326)
angola = gpd.read_file("/Users/inesschwartz/Desktop/GIS_Angola/tiles/ao.shp")
angola = angola.to_crs("EPSG:4326")  # make sure CRS matches rasters


In [3]:
import rioxarray as rxr
import xarray as xr
from pathlib import Path

data_dir = Path("/Users/inesschwartz/GreenDataScience/Thesis/climaticdata/bioclimaticvariables/bio_10m_bil")

bio_files = [data_dir / f"bio{i}.bil" for i in range(1, 20)]
bio_names = {f"bio{i}": f"BIO{i}" for i in range(1, 20)}

datasets = []

for i, file in enumerate(bio_files, 1):
    var_name = bio_names[f"bio{i}"]
    print(f"Loading {file.name} as {var_name}")
    
    da = rxr.open_rasterio(file, masked=True)
    
    # Drop band dimension if present
    if "band" in da.dims:
        da = da.squeeze("band", drop=True)
    
    # Clip to Angola
    da_clip = da.rio.clip(angola.geometry, angola.crs)
    
    # Rename
    da_clip = da_clip.rename(var_name)
    
    datasets.append(da_clip)

# Combine into one Dataset
bio_ds = xr.merge(datasets)
print(bio_ds)

Loading bio1.bil as BIO1
Loading bio2.bil as BIO2
Loading bio3.bil as BIO3
Loading bio4.bil as BIO4
Loading bio5.bil as BIO5
Loading bio6.bil as BIO6
Loading bio7.bil as BIO7
Loading bio8.bil as BIO8
Loading bio9.bil as BIO9
Loading bio10.bil as BIO10
Loading bio11.bil as BIO11
Loading bio12.bil as BIO12
Loading bio13.bil as BIO13
Loading bio14.bil as BIO14
Loading bio15.bil as BIO15
Loading bio16.bil as BIO16
Loading bio17.bil as BIO17
Loading bio18.bil as BIO18
Loading bio19.bil as BIO19
<xarray.Dataset> Size: 462kB
Dimensions:      (x: 74, y: 82)
Coordinates:
  * x            (x) float64 592B 11.75 11.92 12.08 12.25 ... 23.58 23.75 23.92
  * y            (y) float64 656B -4.417 -4.583 -4.75 ... -17.58 -17.75 -17.92
    spatial_ref  int64 8B 0
Data variables: (12/19)
    BIO1         (y, x) float32 24kB nan nan nan nan nan ... nan nan nan nan nan
    BIO2         (y, x) float32 24kB nan nan nan nan nan ... nan nan nan nan nan
    BIO3         (y, x) float32 24kB nan nan nan nan nan .

In [ ]:
/Users/inesschwartz/Desktop/GIS_Angola/tiles/ago_admbnda_gadm_ine_ocha_20180904/ago_admbnda_adm1_gadm_ine_ocha_20180904.shp

In [9]:
import geopandas as gpd
import rioxarray as rxr
import xarray as xr
import pandas as pd

# -----------------------------
# 1. Load districts shapefile
# -----------------------------
districts = gpd.read_file("/Users/inesschwartz/Desktop/GIS_Angola/tiles/ago_admbnda_gadm_ine_ocha_20180904/ago_admbnda_adm1_gadm_ine_ocha_20180904.shp")
districts = districts.to_crs("EPSG:4326")  # make sure CRS matches rasters

# -----------------------------
# 2. Make sure rasters are clipped to Angola (optional if already clipped)
# -----------------------------
# bio_ds = xr.merge([...])  # your clipped raster dataset

# -----------------------------
# 3. Compute district-level mean for each BIO variable
# -----------------------------
results = []

for var in bio_ds.data_vars:
    print(f"Processing {var}...")
    da = bio_ds[var]

    district_means = []

    # Loop over districts
    for idx, row in districts.iterrows():
        # Clip raster to district polygon
        da_clip = da.rio.clip([row.geometry], districts.crs, drop=True, all_touched=True)
        
        # Compute mean of non-NaN pixels
        mean_val = float(da_clip.mean())
        district_means.append(mean_val)
    
    # Store results
    results.append(pd.DataFrame({
        "district": districts["ADM1_EN"],  # replace with your district name column
        var: district_means
    }))

# -----------------------------
# 4. Merge all BIO variables into one table
# -----------------------------
from functools import reduce

district_bio = reduce(lambda left, right: pd.merge(left, right, on="district"), results)
print(district_bio)

# -----------------------------
# 5. Save as CSV for later use
# -----------------------------
district_bio.to_csv(
    "/Users/inesschwartz/GreenDataScience/Thesis/climaticdata/districtclimdata/district_bio.csv",
    index=False
)


Processing BIO1...
Processing BIO2...
Processing BIO3...
Processing BIO4...
Processing BIO5...
Processing BIO6...
Processing BIO7...
Processing BIO8...
Processing BIO9...
Processing BIO10...
Processing BIO11...
Processing BIO12...
Processing BIO13...
Processing BIO14...
Processing BIO15...
Processing BIO16...
Processing BIO17...
Processing BIO18...
Processing BIO19...
          district        BIO1        BIO2       BIO3         BIO4  \
0            Bengo  250.525635   86.115387  62.487179  1524.974365   
1         Benguela  217.027588  107.855171  64.875862  1590.765503   
2              Bié  199.159851  142.780670  61.126392  1697.412598   
3          Cabinda  246.947372   74.315788  55.210526  1827.052612   
4   Cuando Cubango  212.911209  164.218063  59.426792  2987.794434   
5     Cuanza Norte  229.154755  104.261902  67.214287  1443.547607   
6       Cuanza Sul  217.139999  109.285004  65.650002  1441.635010   
7           Cunene  222.320465  158.050186  60.343628  2825.702637   

ImportError: cannot import name 'SequenceNotStr' from 'pandas._typing' (/opt/homebrew/lib/python3.11/site-packages/pandas/_typing.py)

In [12]:
import pandas as pd
print(pd.__version__)  # should be 2.1.x or higher
print(pd.__file__)     # should point to /opt/homebrew/opt/python@3.11/lib/python3.11/site-packages/pandas


2.0.3
/opt/homebrew/lib/python3.11/site-packages/pandas/__init__.py


In [5]:
print(districts.columns)


Index(['Shape_Leng', 'Shape_Area', 'ADM1_EN', 'ADM1_PCODE', 'ADM1_REF',
       'ADM1ALT1EN', 'ADM1ALT2EN', 'ADM0_EN', 'ADM0_PCODE', 'date', 'validOn',
       'validTo', 'geometry'],
      dtype='object')
